In [0]:
# Importing dependency libraries
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#from google.cloud import bigquery
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import math
from ipywidgets import interact, IntSlider
from IPython.display import display
##bigquery_client = bigquery.Client()
import warnings; warnings.simplefilter('ignore')


In [0]:
#defining the thresholds
Threshold_3yrMA=0.1
Threshold_lastyear=0.1
Threshold_lastweek=0.1
Threshold_3weekMA=0.1
Threshold_avgStore=0.1

#### Importing the data

In [0]:
allFiles = glob.glob("../bratin/opr_data_all_store_4yr/*.csv")
#glob uses 
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
df = pd.concat(list_)

In [0]:
df.head()

,locn_nbr,mth_desc,wk_nbr,soar_nm,div_nm,ln_dept_desc,sub_ln_catg_grp_desc,total_quantity,total_sales,total_margin
0,9122,"November,2016",201641,ELECTRONICS,HOME ENTERTAINMENT,MOVIES,SBT BLU-RAY DVD SOFTWARE,1,24.99,5.24
1,1822,"July,2018",201825,APPAREL - RTW / WOMENS,SEASONAL APPAREL,OUTERWEAR,MISSES OUTERWEAR,1,15.99,2.87
2,1217,"December,2017",201744,APPLIANCES,HOME ENVIRONMENT,SUPPLEMENTAL HEATING,PORTABLE ELECTRIC HEATERS,2,78.98,47.09
3,1189,"July,2016",201625,APPLIANCES,AIR & WATER APPLIANCES,ROOM AIR CONDITIONERS,LARGE CAPACITY,3,1659.97,676.04
4,2309,"February,2017",201704,APPLIANCES,HOME ENVIRONMENT,AIR CLEANERS,KENMORE AIR CLEANERS,2,184.98,73.19


In [0]:
df.shape

(46075228, 10)

In [0]:
df.wk_nbr.max()

201909

#### Creating data fragments for faster searching

In [0]:
#Creating a dataframe for the location to be explored
df_locn = df[df.locn_nbr == 7705]

In [0]:
#creating test dataframe with the data for current year and in the specified location
df_locn_2019 = df_locn[df_locn.wk_nbr >= 201901]

In [0]:
#dataframe with current year data across all stores, used for across store comparisons
df_2019 = df[df.wk_nbr >= 201901]

#### Function definitions

In [0]:
ctg_list = pd.read_csv('category_mean_all_store_2019.csv',index_col=None)

'''
Rolling average gives us the mean of fixed number of weeks defined by the window. 
name - Name of the category for which rolling average is needed.
wk_nbr - week number for which the rolling average needs to be calculated
metric - the metric for which the rolling average is calculated. Options are 'total_sales', 'total_margin' & 'total_quantity'.
window - the window length for which the mean has to be calculated. it denotes the number of entries across which the mean will be calculated.
'''

def rolling_avg(name,wk_nbr,metric,window):
    return df_locn[(df_locn.sub_ln_catg_grp_desc == name)&(df_locn.wk_nbr<wk_nbr)].sort_values(['wk_nbr'],ascending=False)[:window][metric].mean()

'''
Value for week function returns the value for a particular week. Serves as an getter function
name - name of the category for which value is needed
wk_nbr - week number for value selection
metric = the metric for which value has to be fetched
'''

def value_for_week(name,wk_nbr,metric):
    try:
        return df_locn[(df_locn.sub_ln_catg_grp_desc == name)&(df_locn.wk_nbr==wk_nbr)][metric].values[0]
    except:
        return 0.0

'''
Mean of metrics for a category across all stores.
name - name of the category for which mean is needed
wk_nbr - week number for which all store mean needs to be calculated
metric = the metric for which all store mean is calculated
'''
def avg_perStoreCtg(name,wk_nbr,metric):
    try:
        return df_2019[(df_2019.sub_ln_catg_grp_desc == name)&(df_2019.wk_nbr==wk_nbr)][metric].mean()
    except:
        return np.nan
    
'''Change compared to the week last year.
name - name of the category for which change is to be calculated
wk_nbr - week number for which change needs to be calculated
metric = the metric for which change has to be calculated
'''

def change_wow_lastYear(name,wk_nbr,metric):
    try:
        return 1 - (value_for_week(name=name,wk_nbr=wk_nbr-100,metric=metric) / value_for_week(name=name,wk_nbr=wk_nbr,metric=metric))
    except:
        return np.nan

'''Change compared to the previous week.
name - name of the category for which change is to be calculated
wk_nbr - week number for which change needs to be calculated
metric = the metric for which change has to be calculated
'''
def change_wow(name,wk_nbr,metric):
    try:
        if wk_nbr%100 == 1:
            return 1 - (value_for_week(name=name,wk_nbr=wk_nbr-49,metric=metric) / value_for_week(name=name,wk_nbr=wk_nbr,metric=metric))
        else: 
            return 1 - (value_for_week(granularity = granularity,name=name,wk_nbr=wk_nbr-1,metric=metric,locn_nbr=locn_nbr) / value_for_week(granularity = granularity,name=name,wk_nbr=wk_nbr,metric=metric,locn_nbr=locn_nbr))
    except:
        return np.nan 

'''
Change compared to 3 year moving average.
name - name of the category for which change is to be calculated
wk_nbr - week number for which change needs to be calculated
metric = the metric for which change has to be calculated
'''
    
def change_wo3yMA(name,wk_nbr,metric):
    try:
        return 1 - (rolling_avg(name=name,wk_nbr=wk_nbr,metric=metric,window=156) / value_for_week(name=name,wk_nbr=wk_nbr,metric=metric))
    except:
        return np.nan
'''
Change compared to the 3 week .
name - name of the category for which change is to be calculated
wk_nbr - week number for which change needs to be calculated
metric = the metric for which change has to be calculated
'''

def change_wo3wMA(name,wk_nbr,metric):
    try:
        return 1 - (rolling_avg(name=name,wk_nbr=wk_nbr,metric=metric,window=3) / value_for_week(name=name,wk_nbr=wk_nbr,metric=metric))
    except:
        return np.nan
'''
Change compared to mean of the given metric across all stores .
name - name of the category for which change is to be calculated
wk_nbr - week number for which change needs to be calculated
metric = the metric for which change has to be calculated
'''

def change_all_store_mean(mean,metric_value):
    try:
        return 1 - (mean/ metric_value)
    except:
        return np.nan
'''
Calculates the comparison columns for the anomaly detection
name - name of the category for which change is to be calculated
wk_nbr - week number for which change needs to be calculated
metric = the metric for which change has to be calculated
'''

def anomaly_condition_compute(name,wk_nbr,metric):
    wk_rollingAvg_3yr = rolling_avg(name=name,wk_nbr=wk_nbr,metric=metric,window=156)
    change_3yrRollingAvg = change_wo3yMA( name=name,wk_nbr=wk_nbr,metric=metric ) * 100
    wk_last_yr = value_for_week( name=name,wk_nbr=wk_nbr-100,metric=metric )
    change_wk_last_yr = change_wow_lastYear( name=name,wk_nbr=wk_nbr,metric=metric ) * 100
    last_wk = value_for_week( name=name,wk_nbr=(wk_nbr-1) if (wk_nbr%100 != 1) else (wk_nbr-49),metric=metric )
    change_last_3wkMA = change_wo3wMA(name=name,wk_nbr=wk_nbr,metric=metric) * 100
    all_store_mean = ctg_list[(ctg_list.sub_ln_catg_grp_desc == name)&(ctg_list.wk_nbr==wk_nbr)]['all_store_mean_'+metric[6:]].values[0]
    
    return pd.Series([wk_rollingAvg_3yr,change_3yrRollingAvg,wk_last_yr,change_wk_last_yr,last_wk,change_last_3wkMA,all_store_mean])

'''
Filtering function based on predefined threshold
'''
    
def anomalyRule1(row,metric,threshold_3yrMA=0.1,threshold_lastyear=0.1,threshold_lastweek=0.1,threshold_3weekMA=0.1,threshold_avgStore=0.1):
    condition1 = abs(row['change%_'+metric+'_wk_last_yr']) > (threshold_lastyear*100)
    condition2 = abs(row['change%_3yrRollingAvg_'+metric])> (threshold_3yrMA*100)
    condition3 = abs(row['change%_'+metric+'_last_wk']) > (threshold_lastweek*100)
    condition4 = abs(row['change%_'+metric+'_last_3wkMA']) > (threshold_3weekMA*100)
    condition5 =abs(row['change%_avg_storeBU']) > (threshold_avgStore*100)
    return (condition1 & condition2 & condition3 & condition4 & condition5)

## Category Level Detection
### Rule 1: 
If 
- a) the week over week sales of a category group changes by 10% (+ve or -ve) or more, and 
- b) compared to the sales of the same week last year, and 
- c) compared to the average of last 3 years sales in that same week
- d) compared to the average of previous 3 weeks
- e) compared to the average for that week across all stores
<br>
then flag the week as an anomaly. 

For example: if 
- a) Week 6, 2019 sales in Apparel Mens Activewear Category group is \\$79K as compared to week 5, 2019 sales in the same category group was \\$35K and 
- b) sales in week 6, 2018 in the same category group was \\$40K and 
- c) average of sales in last 3 yrs (Week 6 in 2016 - Week 5 in 2019) was \\$48K - 
- d) average of previous 3 weeks (Week 3 in 2019 - Week 5 in 2019) was \\$42K
- e) average for week 6 2019 across all stores is \\$26K
<br>
then flag week 6, 2019 sales in Apparel Mens Activewear Category group as an anomaly. 

In [0]:
#define the metric for the detection
metric='total_sales'
#compute the comparison columns
newcols = df_locn_2019.apply(lambda row: anomaly_condition_compute(name=row.sub_ln_catg_grp_desc,wk_nbr=row.wk_nbr,metric=metric), axis=1)

In [0]:
#join the computed comparison columns to the test dataframe
newcols.columns = ['3yr_rollingAvg_wk_'+metric, 'change%_3yrRollingAvg_'+metric , metric+'_wk_last_yr','change%_'+metric+'_wk_last_yr',metric+'_last_wk','change%_'+metric+'_last_3wkMA','all_store_mean']
calculated_df_sales = df_locn_2019.join(newcols) 

In [0]:
#compute the comparison for the average across stores change and change from last week
calculated_df_sales['change%_avg_storeBU'] = calculated_df_sales.apply(lambda row: change_all_store_mean(row['all_store_mean'],row[metric]),axis=1) * 100

calculated_df_sales['change%_'+metric+'_last_wk']=calculated_df_sales.apply(lambda row: change_wow( name=row.sub_ln_catg_grp_desc,wk_nbr=row.wk_nbr,metric=metric ), axis=1)*100

In [0]:
calculated_df_sales.shape

(5133, 19)

In [0]:
#dropping the rows with NAN owing to missing data
calculated_df_sales.dropna(inplace=True)

In [0]:
calculated_df_sales.shape

(591, 19)

In [0]:
calculated_df_sales.head()

,locn_nbr,mth_desc,wk_nbr,soar_nm,div_nm,ln_dept_desc,sub_ln_catg_grp_desc,total_quantity,total_sales,total_margin,3yr_rollingAvg_wk_total_sales,change%_3yrRollingAvg_total_sales,total_sales_wk_last_yr,change%_total_sales_wk_last_yr,total_sales_last_wk,change%_total_sales_last_3wkMA,all_store_mean,change%_avg_storeBU,change%_total_sales_last_wk
11296,7705,"February,2019",201901,SEASONAL,SEASONAL,VALENTINES,PLUSH,2924,11304.91,606.56,3004.92,73.42,109.42,99.03,5762.29,76.11,222.11,98.04,49.03
13718,7705,"February,2019",201901,SUPPORT UNITS,REGIONAL MERCHANDISE,REGIONAL MERCHANDISE,REGIONAL MERCHANDISE,215,1868.65,932.90,811.36,56.58,166.90,91.07,1920.56,44.40,844.67,54.80,-2.78
26424,7705,"February,2019",201901,DRUG STORE,"CARDS, WRAP & PARTY",GREETING CARDS,SBT AG WRAP,699,2257.98,1125.28,2073.25,8.18,2290.16,-1.43,1562.14,25.88,354.68,84.29,30.82
58859,7705,"February,2019",201901,GROCERY & HOUSEHOLD,TOBACCO & ALCOHOL,CIGARETTES/TOBACCO,CIGARETTES,281,3820.04,550.10,6540.90,-71.23,6885.71,-80.25,7006.16,-70.41,303.59,92.05,-83.41
62956,7705,"February,2019",201901,APPAREL - RTW / WOMENS,RTW,MISSY SPORTSWEAR,CLASSIC ELEMENTS,1,7.98,4.08,31.20,-290.99,0.00,100.00,0.00,-195.07,13.47,-68.76,100.00


In [0]:
#applying the threshold rule for getting the anomalies
rule1Anomaly = calculated_df_sales[calculated_df_sales.apply(lambda row: anomalyRule1(row=row,metric='total_sales',threshold_3yrMA=Threshold_3yrMA,threshold_lastyear=Threshold_lastyear,threshold_lastweek=Threshold_lastweek,threshold_3weekMA=Threshold_3weekMA,threshold_avgStore=Threshold_avgStore), axis=1)].copy()

In [0]:
rule1Anomaly.shape

(302, 19)

In [0]:
del rule1Anomaly['locn_nbr']
del rule1Anomaly['mth_desc']

In [0]:
freeze_header(rule1Anomaly.sort_values(by=['total_sales'],ascending=False).reset_index(drop=True),num_columns=10)

interactive(children=(IntSlider(value=20, description='rows', max=302, min=20, readout=False), IntSlider(value…

### Rule 4: 
If a) the week over week sales margin of a category group changes by 50% (+ve or -ve) or more, and b) compared to the sales margin of the same week last year, and c) compared to the average of last 3 years sales margin in that same week - then flag the week as an anomaly. For example: if a) Week 9, 2019 sales margin in Seasonal Category group is $680.64 as compared to week 8, 2019 sales in the same category group was $1200.89 and b) sales margin in week 9, 2018 in the same category group was $1500.78 and c) average of sales margin in last 3 yrs (Week 9 in 2016 - Week 8 in 2018) was $1300.56 - then flag week 9, 2019 sales margin in Seasonal Category group as an anomaly.

In [0]:
def freeze_header(df, num_rows=20, num_columns=10, step_rows=1,step_columns=1): 
    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]), max=df.shape[0], step=step_rows, description='rows', readout=False, disabled=False, continuous_update=True, orientation='horizontal', slider_color='purple'), last_column=IntSlider(min=min(num_columns, df.shape[1]), max=df.shape[1], step=step_columns, description='columns', readout=False, disabled=False, continuous_update=True, orientation='horizontal', slider_color='purple')) 
    def freezeheader(last_row, last_column): 
        display(df.iloc[max(0, last_row-num_rows):last_row, max(0, last_column-num_columns):last_column])